# Importing the libraries

# New Section

In [ ]:
!git clone https://github.com/mandar196/Hate_Speech_Detection-NLP.git

Cloning into 'Hate_Speech_Detection-NLP'...
remote: Enumerating objects: 19, done.
remote: Counting objects: 100% (19/19), done.
remote: Compressing objects: 100% (18/18), done.
remote: Total 19 (delta 4), reused 0 (delta 0), pack-reused 0
Unpacking objects: 100% (19/19), done.


In [ ]:
!gdown --id 1ti6QbZh_j9rh9fhKGZIoThM7FN9tF3Un #training data

/usr/local/lib/python3.7/dist-packages/gdown/cli.py:131: FutureWarning: Option `--id` was deprecated in version 4.3.1 and will be removed in 5.0. You don't need to pass it anymore to use a file ID.
  category=FutureWarning,
Downloading...
From: https://drive.google.com/uc?id=1ti6QbZh_j9rh9fhKGZIoThM7FN9tF3Un
To: /content/labeled_data.csv
100% 2.57M/2.57M [00:00<00:00, 113MB/s]


In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import matplotlib.pyplot as plt
from wordcloud import WordCloud

#to data preprocessing
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import OneHotEncoder

#NLP tools
import re
import nltk
nltk.download('stopwords')
from nltk.corpus import stopwords
from nltk.stem.porter import PorterStemmer
from sklearn.feature_extraction.text import CountVectorizer

#train split and fit models
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.neighbors import KNeighborsClassifier
from sklearn import svm
from xgboost import XGBClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.naive_bayes import GaussianNB

#model selection
from sklearn.metrics import confusion_matrix, accuracy_score


# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 5GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


# Importing the dataset

In [ ]:
dataset = pd.read_csv('/content/stormfront.csv')
dataset.head()

,Unnamed: 0,file_id,label,tweet
0,0,12834217_1,noHate,"['As of March 13th , 2014 , the booklet had be..."
1,1,12834217_2,noHate,['In order to help increase the booklets downl...
2,2,12834217_3,noHate,['( Simply copy and paste the following text i...
3,3,12834217_4,hate,['Click below for a FREE download of a colorfu...
4,4,12834217_5,noHate,"[""Click on the `` DOWNLOAD ( 7.42 MB ) '' gree..."


In [ ]:
print(dataset['label'].unique())

['noHate' 'hate' 'idk/skip' 'relation']


In [ ]:
dataset.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 10944 entries, 0 to 10943
Data columns (total 4 columns):
 #   Column      Non-Null Count  Dtype 
---  ------      --------------  ----- 
 0   Unnamed: 0  10944 non-null  int64 
 1   file_id     10944 non-null  object
 2   label       10944 non-null  object
 3   tweet       10944 non-null  object
dtypes: int64(1), object(3)
memory usage: 342.1+ KB


In [ ]:
dataset.describe().T

,count,mean,std,min,25%,50%,75%,max
Unnamed: 0,10944.0,5471.5,3159.405007,0.0,2735.75,5471.5,8207.25,10943.0


In [ ]:
dataset['label'] = dataset['label'].replace({'hate': 0})
dataset['label'] = dataset['label'].replace({'noHate': 1})
dataset['label'] = dataset['label'].replace({'relation': 1})
dataset['label'] = dataset['label'].replace({'idk/skip': 1})

In [ ]:
dicti = { 'label': 'class','tweet': 'tweet'}
dataset.rename(columns=dicti,
          inplace=True)

In [ ]:
dt_trasformed = dataset[['class', 'tweet']]
xy = dt_trasformed.iloc[:, :-1].values
print(xy)

[[1]
 [1]
 [1]
 ...
 [1]
 [0]
 [1]]


In [ ]:
xy = np.ravel(xy,order='C')

### Encoding the Dependent Variable

In [ ]:
# ct = ColumnTransformer(transformers=[('encoder', OneHotEncoder(), [0])], remainder='passthrough')
# y = np.array(ct.fit_transform(y))

In [ ]:
print(y)

[[0. 0. 1.]
 [1. 0. 0.]
 [1. 0. 0.]
 ...
 [1. 0. 0.]
 [0. 0. 1.]
 [0. 0. 1.]]


I separated this y in two variables that we will use to fit hate speech models and offensive speech models

In [ ]:
# y_df = pd.DataFrame(y)
# y_hate = np.array(y_df[0])
# y_offensive = np.array(y_df[1])
# # y_neither = np.array(y_df[2])
# print(sum(y_hate==0))

9748


In [ ]:
# print(y_hate)
# print(y_offensive)

[0. 0. 0. ... 0. 1. 0.]
[1. 1. 1. ... 1. 0. 1.]


## Cleaning the texts

In [ ]:
corpus = []
for i in range(0, 10944):
  review = re.sub('[^a-zA-Z]', ' ', dt_trasformed['tweet'][i])
  review = review.lower()
  review = review.split()
  ps = PorterStemmer()
  all_stopwords = stopwords.words('english')
  all_stopwords.remove('not')
  review = [ps.stem(word) for word in review if not word in set(all_stopwords)]
  review = ' '.join(review)
  corpus.append(review)

In [ ]:
cv = CountVectorizer(max_features = 2000)
X = cv.fit_transform(corpus).toarray()

## Splitting the dataset into the Training set and Test set

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(X, xy, test_size = 0.30, random_state = 0)

In [ ]:
# dfd = pd.DataFrame(y_test)
# dfd.to_csv('ML_SF_ytest.csv')
# # print(type(data))
# dfdd = pd.DataFrame(X_test)
# dfdd.to_csv('ML_SF_Xtest.csv')

## Finding the best models to predict hate speech

**Naive Bayes**

In [ ]:
classifier_np = GaussianNB()
classifier_np.fit(X_train, y_train)

GaussianNB()

**Decision Tree**

In [ ]:
classifier_dt = DecisionTreeClassifier(criterion = 'entropy', random_state = 0)
classifier_dt.fit(X_train, y_train)

DecisionTreeClassifier(criterion='entropy', random_state=0)

**KNN**

In [ ]:
classifier_knn = KNeighborsClassifier(n_neighbors = 5, metric = 'minkowski', p = 2)
classifier_knn.fit(X_train, y_train)

KNeighborsClassifier()

**Logistic Regression**

In [ ]:
classifier_lr = LogisticRegression(random_state = 0)
classifier_lr.fit(X_train, y_train)

LogisticRegression(random_state=0)

**Random Forest**

In [ ]:
classifier_rf = RandomForestClassifier(n_estimators = 10, criterion = 'entropy', random_state = 0)
classifier_rf.fit(X_train, y_train)

RandomForestClassifier(criterion='entropy', n_estimators=10, random_state=0)

SVM Classifier

In [ ]:
classifier_svm = svm.SVC()
classifier_svm.fit(X_train, y_train)

SVC()

XGBoost Classifier

In [ ]:
classifier_xgb= XGBClassifier()
classifier_xgb.fit(X_train, y_train)

XGBClassifier()

# Making the Confusion Matrix for each model

In [ ]:
from sklearn.metrics import classification_report

In [ ]:
#Naive Bayes
y_pred_np = classifier_np.predict(X_test)
cm = confusion_matrix(y_test, y_pred_np)
print(classification_report(y_test,y_pred_np))
print(y_pred_np)
print(cm)


              precision    recall  f1-score   support

           0       0.12      0.68      0.20       342
           1       0.91      0.40      0.55      2942

    accuracy                           0.43      3284
   macro avg       0.52      0.54      0.38      3284
weighted avg       0.83      0.43      0.52      3284

[0 1 0 ... 1 0 1]
[[ 233  109]
 [1774 1168]]


In [ ]:
#KNN
y_pred_knn = classifier_knn.predict(X_test)
cm = confusion_matrix(y_test, y_pred_knn)
print(classification_report(y_test,y_pred_knn))
print(cm)

              precision    recall  f1-score   support

           0       0.44      0.02      0.04       342
           1       0.90      1.00      0.94      2942

    accuracy                           0.90      3284
   macro avg       0.67      0.51      0.49      3284
weighted avg       0.85      0.90      0.85      3284

[[   7  335]
 [   9 2933]]


In [ ]:
#XGBoost Classifier
y_pred_xgb = classifier_xgb.predict(X_test)
cm = confusion_matrix(y_test, y_pred_xgb)
print(classification_report(y_test,y_pred_xgb))
print(cm)

              precision    recall  f1-score   support

           0       0.74      0.07      0.12       342
           1       0.90      1.00      0.95      2942

    accuracy                           0.90      3284
   macro avg       0.82      0.53      0.54      3284
weighted avg       0.89      0.90      0.86      3284

[[  23  319]
 [   8 2934]]


In [ ]:
#SVM
y_pred_svm = classifier_svm.predict(X_test)
cm = confusion_matrix(y_test, y_pred_svm)
print(classification_report(y_test,y_pred_svm))
print(cm)

              precision    recall  f1-score   support

           0       0.70      0.04      0.08       342
           1       0.90      1.00      0.95      2942

    accuracy                           0.90      3284
   macro avg       0.80      0.52      0.51      3284
weighted avg       0.88      0.90      0.86      3284

[[  14  328]
 [   6 2936]]


In [ ]:
#Logistic Regression
y_pred_lr=classifier_lr.predict(X_test)
cm = confusion_matrix(y_test, y_pred_lr)
print(classification_report(y_test,y_pred_lr))
print(cm)

              precision    recall  f1-score   support

           0       0.57      0.28      0.38       342
           1       0.92      0.98      0.95      2942

    accuracy                           0.90      3284
   macro avg       0.75      0.63      0.66      3284
weighted avg       0.89      0.90      0.89      3284

[[  96  246]
 [  71 2871]]


In [ ]:
#Decision Tree
y_pred_dt = classifier_dt.predict(X_test)
cm = confusion_matrix(y_test, y_pred_dt)
print(classification_report(y_test,y_pred_dt))
print(cm)


              precision    recall  f1-score   support

           0       0.36      0.36      0.36       342
           1       0.93      0.92      0.92      2942

    accuracy                           0.87      3284
   macro avg       0.64      0.64      0.64      3284
weighted avg       0.87      0.87      0.87      3284

[[ 124  218]
 [ 224 2718]]


In [ ]:
#Random Florest
y_pred_rf = classifier_rf.predict(X_test)
cm = confusion_matrix(y_test, y_pred_rf)
print(classification_report(y_test,y_pred_rf))
print(cm)

              precision    recall  f1-score   support

           0       0.51      0.20      0.29       342
           1       0.91      0.98      0.94      2942

    accuracy                           0.90      3284
   macro avg       0.71      0.59      0.62      3284
weighted avg       0.87      0.90      0.88      3284

[[  68  274]
 [  65 2877]]


In [ ]:
rf_score = accuracy_score(y_test, y_pred_rf)
knn_score = accuracy_score(y_test, y_pred_knn)
svm_score = accuracy_score(y_test, y_pred_svm)
xgb_score = accuracy_score(y_test, y_pred_xgb)
lr_score = accuracy_score(y_test, y_pred_lr)
dt_score = accuracy_score(y_test, y_pred_dt)
np_score = accuracy_score(y_test, y_pred_np)

print('Random Forest Accuracy: ', str(rf_score))
print('K Nearest Neighbours Accuracy: ', str(knn_score))
print('Support Vector Machine Accuracy: ', str(svm_score))
print('XGBoost Classifier Accuracy: ', str(xgb_score))
print('Logistic Regression Accuracy: ',str(lr_score))
print('Decision Tree Accuracy: ', str(dt_score))
print('Naive Bayes Accuracy: ', str(np_score))

Random Forest Accuracy:  0.8967722289890377
K Nearest Neighbours Accuracy:  0.8952496954933008
Support Vector Machine Accuracy:  0.8982947624847747
XGBoost Classifier Accuracy:  0.9004263093788063
Logistic Regression Accuracy:  0.9034713763702802
Decision Tree Accuracy:  0.8654080389768575
Naive Bayes Accuracy:  0.4266138855054811


So XGBoost Classifier looks better to predict hate speech based on this dataset. It's important to emphasize Random Forest,KNN and Decision Tree had great results as well. This Dataset looks like a product of artificial intelligence to classify hate and offensive speech.

In [ ]:
#Now finally saving the model with XGB Classifier.
import joblib
joblib.dump(classifier_xgb, 'hatespeech.pkl')

['hatespeech.pkl']

In [ ]:
from sklearn.metrics import precision_recall_fscore_support

In [ ]:
score_rf=precision_recall_fscore_support(y_test,y_pred_rf,average='weighted')
print(score_rf)

(0.8712031229904448, 0.8967722289890377, 0.8758325611967008, None)


In [ ]:
score_knn=precision_recall_fscore_support(y_test,y_pred_knn,average='weighted')
print(score_knn)

(0.8495867821729777, 0.8952496954933008, 0.8503056057382509, None)


In [ ]:
score_svm=precision_recall_fscore_support(y_test,y_pred_svm,average='weighted')
print(score_svm)

(0.8787325953523919, 0.8982947624847747, 0.8556997195953079, None)


In [ ]:
score_xgb=precision_recall_fscore_support(y_test,y_pred_xgb,average='weighted')
print(score_xgb)

(0.8852739433373252, 0.9004263093788063, 0.8614144135245133, None)


In [ ]:
score_lr=precision_recall_fscore_support(y_test,y_pred_lr,average='weighted')
print(score_lr)

(0.8850213640284164, 0.9034713763702802, 0.8882715574157422, None)


In [ ]:
score_dt=precision_recall_fscore_support(y_test,y_pred_dt,average='weighted')
print(score_dt)

(0.8664484058703787, 0.8654080389768575, 0.8659245286833444, None)


In [ ]:
score_np=precision_recall_fscore_support(y_test,y_pred_np,average='weighted')
print(score_np)

(0.8314816656472708, 0.4266138855054811, 0.5166839509135552, None)


# Testing

In [ ]:
from google.colab import drive
drive.mount('/content/gdrive')

Mounted at /content/gdrive


## Test on Large Facebook

In [ ]:
d = pd.read_csv('/content/gdrive/MyDrive/Colab Notebooks/ML_TEST/Data/ML_FB!_Xtest.csv')
# d.head(5)
d.columns.values[0] = 'id'
d.drop('id', inplace=True, axis=1)
X_t = d.to_numpy()
de = pd.read_csv('/content/gdrive/MyDrive/Colab Notebooks/ML_TEST/Data/ML_FB!_ytest.csv')
# d.head(5)
de.columns.values[0] = 'id'
de.drop('id', inplace=True, axis=1)
y_t = de.to_numpy()
# X_t

In [ ]:
#Naive Bayes by Davidson or itself
y_pred_np = classifier_np.predict(X_t)
cm = confusion_matrix(y_t, y_pred_np)
print(classification_report(y_t,y_pred_np))
print(cm)
score_np=precision_recall_fscore_support(y_t,y_pred_np,average='weighted')
print(score_np)
score_np=precision_recall_fscore_support(y_t,y_pred_np,average='micro')
print(score_np)
score_np=precision_recall_fscore_support(y_t,y_pred_np,average='macro')
print(score_np)

              precision    recall  f1-score   support

           0       0.34      0.24      0.28       929
           1       0.63      0.73      0.67      1621

    accuracy                           0.55      2550
   macro avg       0.48      0.49      0.48      2550
weighted avg       0.52      0.55      0.53      2550

[[ 226  703]
 [ 440 1181]]
(0.5221108463277815, 0.5517647058823529, 0.5316267001409698, None)
(0.5517647058823529, 0.5517647058823529, 0.5517647058823529, None)
(0.4830985444042769, 0.48591747575716726, 0.4786400082282812, None)


In [ ]:
#KNN
y_pred_knn = classifier_knn.predict(X_t)
cm = confusion_matrix(y_t, y_pred_knn)
print(classification_report(y_t,y_pred_knn))
print(cm)
score_knn=precision_recall_fscore_support(y_t,y_pred_knn,average='weighted')
print(score_knn)
score_knn=precision_recall_fscore_support(y_t,y_pred_knn,average='micro')
print(score_knn)
score_knn=precision_recall_fscore_support(y_t,y_pred_knn,average='macro')
print(score_knn)

              precision    recall  f1-score   support

           0       0.00      0.00      0.00       929
           1       0.64      1.00      0.78      1621

    accuracy                           0.64      2550
   macro avg       0.32      0.50      0.39      2550
weighted avg       0.40      0.64      0.49      2550

[[   0  929]
 [   0 1621]]
(0.4040970396001538, 0.635686274509804, 0.49410091152260477, None)
(0.635686274509804, 0.635686274509804, 0.635686274509804, None)
(0.317843137254902, 0.5, 0.38863581874850156, None)


/usr/local/lib/python3.7/dist-packages/sklearn/metrics/_classification.py:1318: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.7/dist-packages/sklearn/metrics/_classification.py:1318: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.7/dist-packages/sklearn/metrics/_classification.py:1318: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.7/dist-packages/sklearn/metrics/_classificat

In [ ]:
#XGBoost Classifier
y_pred_xgb = classifier_xgb.predict(X_t)
cm = confusion_matrix(y_t, y_pred_xgb)
print(classification_report(y_t,y_pred_xgb))
print(cm)
score_xgb=precision_recall_fscore_support(y_t,y_pred_xgb,average='weighted')
print(score_xgb)
score_xgb=precision_recall_fscore_support(y_t,y_pred_xgb,average='micro')
print(score_xgb)
score_xgb=precision_recall_fscore_support(y_t,y_pred_xgb,average='macro')
print(score_xgb)

              precision    recall  f1-score   support

           0       0.25      0.00      0.00       929
           1       0.64      1.00      0.78      1621

    accuracy                           0.63      2550
   macro avg       0.44      0.50      0.39      2550
weighted avg       0.50      0.63      0.49      2550

[[   1  928]
 [   3 1618]]
(0.4950613033901699, 0.6349019607843137, 0.49444084662370186, None)
(0.6349019607843137, 0.6349019607843137, 0.6349019607843137, None)
(0.44275333857030635, 0.4996128584130914, 0.3893607482462496, None)


In [ ]:
#SVM
y_pred_svm = classifier_svm.predict(X_t)
cm = confusion_matrix(y_t, y_pred_svm)
print(classification_report(y_t,y_pred_svm))
print(cm)
score_svm=precision_recall_fscore_support(y_t,y_pred_svm,average='weighted')
print(score_svm)
score_svm=precision_recall_fscore_support(y_t,y_pred_svm,average='micro')
print(score_svm)
score_svm=precision_recall_fscore_support(y_t,y_pred_svm,average='macro')
print(score_svm)

              precision    recall  f1-score   support

           0       0.17      0.00      0.00       929
           1       0.64      1.00      0.78      1621

    accuracy                           0.63      2550
   macro avg       0.40      0.50      0.39      2550
weighted avg       0.46      0.63      0.49      2550

[[   1  928]
 [   5 1616]]
(0.46451966950302126, 0.6341176470588236, 0.49406572468559606, None)
(0.6341176470588236, 0.6341176470588236, 0.6341176470588236, None)
(0.40094339622641506, 0.49899595526688534, 0.38906471679580923, None)


In [ ]:
#Logistic Regression
y_pred_lr=classifier_lr.predict(X_t)
cm = confusion_matrix(y_t, y_pred_lr)
print(classification_report(y_t,y_pred_lr))
print(cm)
score_lr=precision_recall_fscore_support(y_t,y_pred_lr,average='weighted')
print(score_lr)
score_lr=precision_recall_fscore_support(y_t,y_pred_lr,average='micro')
print(score_lr)
score_lr=precision_recall_fscore_support(y_t,y_pred_lr,average='macro')
print(score_lr)

              precision    recall  f1-score   support

           0       0.60      0.04      0.08       929
           1       0.64      0.98      0.78      1621

    accuracy                           0.64      2550
   macro avg       0.62      0.51      0.43      2550
weighted avg       0.63      0.64      0.52      2550

[[  38  891]
 [  25 1596]]
(0.6276882102526039, 0.6407843137254902, 0.5218523659424507, None)
(0.6407843137254902, 0.6407843137254902, 0.6407843137254902, None)
(0.6224558178719819, 0.5127408097036408, 0.4268166755661651, None)


In [ ]:
#Decision Tree
y_pred_dt = classifier_dt.predict(X_t)
cm = confusion_matrix(y_t, y_pred_dt)
print(classification_report(y_t,y_pred_dt))
print(cm)
score_dt=precision_recall_fscore_support(y_t,y_pred_dt,average='weighted')
print(score_dt)
score_dt=precision_recall_fscore_support(y_t,y_pred_dt,average='micro')
print(score_dt)
score_dt=precision_recall_fscore_support(y_t,y_pred_dt,average='macro')
print(score_dt)

              precision    recall  f1-score   support

           0       0.48      0.16      0.24       929
           1       0.65      0.90      0.76      1621

    accuracy                           0.63      2550
   macro avg       0.57      0.53      0.50      2550
weighted avg       0.59      0.63      0.57      2550

[[ 149  780]
 [ 162 1459]]
(0.5887749578820627, 0.6305882352941177, 0.5681052885135335, None)
(0.6305882352941177, 0.6305882352941177, 0.6305882352941177, None)
(0.565364935253307, 0.5302246018849744, 0.4981405649339796, None)


In [ ]:
#Random Florest
y_pred_rf = classifier_rf.predict(X_t)
cm = confusion_matrix(y_t, y_pred_rf)
print(classification_report(y_t,y_pred_rf))
print(cm)
score_rf=precision_recall_fscore_support(y_t,y_pred_rf,average='weighted')
print(score_rf)
score_rf=precision_recall_fscore_support(y_t,y_pred_rf,average='micro')
print(score_rf)
score_rf=precision_recall_fscore_support(y_t,y_pred_rf,average='macro')
print(score_rf)

              precision    recall  f1-score   support

           0       0.53      0.12      0.20       929
           1       0.65      0.94      0.77      1621

    accuracy                           0.64      2550
   macro avg       0.59      0.53      0.49      2550
weighted avg       0.61      0.64      0.56      2550

[[ 115  814]
 [ 100 1521]]
(0.6089463476840365, 0.6415686274509804, 0.5620612844400984, None)
(0.6415686274509804, 0.6415686274509804, 0.6415686274509804, None)
(0.5931377919426324, 0.5310493529157472, 0.48500374751638653, None)


## Test on Facebook

In [ ]:
d = pd.read_csv('/content/gdrive/MyDrive/Colab Notebooks/ML_TEST/Data/ML_FB_Xtest.csv')
# d.head(5)
d.columns.values[0] = 'id'
d.drop('id', inplace=True, axis=1)
X_t = d.to_numpy()
de = pd.read_csv('/content/gdrive/MyDrive/Colab Notebooks/ML_TEST/Data/ML_FB_ytest.csv')
# d.head(5)
de.columns.values[0] = 'id'
de.drop('id', inplace=True, axis=1)
y_t = de.to_numpy()
# X_t

In [ ]:
#Naive Bayes by Davidson or itself
y_pred_np = classifier_np.predict(X_t)
cm = confusion_matrix(y_t, y_pred_np)
print(classification_report(y_t,y_pred_np))
print(cm)
score_np=precision_recall_fscore_support(y_t,y_pred_np,average='weighted')
print(score_np)

              precision    recall  f1-score   support

           0       0.41      0.12      0.18        59
           1       0.61      0.89      0.72        90

    accuracy                           0.58       149
   macro avg       0.51      0.50      0.45       149
weighted avg       0.53      0.58      0.51       149

[[ 7 52]
 [10 80]]
(0.5291246455873381, 0.5838926174496645, 0.508277086694607, None)


In [ ]:
#KNN
y_pred_knn = classifier_knn.predict(X_t)
cm = confusion_matrix(y_t, y_pred_knn)
print(classification_report(y_t,y_pred_knn))
print(cm)
score_knn=precision_recall_fscore_support(y_t,y_pred_knn,average='weighted')
print(score_knn)

              precision    recall  f1-score   support

           0       0.00      0.00      0.00        59
           1       0.60      0.99      0.75        90

    accuracy                           0.60       149
   macro avg       0.30      0.49      0.37       149
weighted avg       0.36      0.60      0.45       149

[[ 0 59]
 [ 1 89]]
(0.3632323598766552, 0.5973154362416108, 0.45175117026676437, None)


In [ ]:
#XGBoost Classifier
y_pred_xgb = classifier_xgb.predict(X_t)
cm = confusion_matrix(y_t, y_pred_xgb)
print(classification_report(y_t,y_pred_xgb))
print(cm)
score_xgb=precision_recall_fscore_support(y_t,y_pred_xgb,average='weighted')
print(score_xgb)

              precision    recall  f1-score   support

           0       1.00      0.08      0.16        59
           1       0.62      1.00      0.77        90

    accuracy                           0.64       149
   macro avg       0.81      0.54      0.46       149
weighted avg       0.77      0.64      0.53       149

[[ 5 54]
 [ 0 90]]
(0.7734899328859061, 0.6375838926174496, 0.5265068404749613, None)


In [ ]:
#SVM
y_pred_svm = classifier_svm.predict(X_t)
cm = confusion_matrix(y_t, y_pred_svm)
print(classification_report(y_t,y_pred_svm))
print(cm)
score_svm=precision_recall_fscore_support(y_t,y_pred_svm,average='weighted')
print(score_svm)

              precision    recall  f1-score   support

           0       0.00      0.00      0.00        59
           1       0.60      1.00      0.75        90

    accuracy                           0.60       149
   macro avg       0.30      0.50      0.38       149
weighted avg       0.36      0.60      0.45       149

[[ 0 59]
 [ 0 90]]
(0.3648484302508896, 0.6040268456375839, 0.45491561596136026, None)


/usr/local/lib/python3.7/dist-packages/sklearn/metrics/_classification.py:1318: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.7/dist-packages/sklearn/metrics/_classification.py:1318: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.7/dist-packages/sklearn/metrics/_classification.py:1318: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.7/dist-packages/sklearn/metrics/_classificat

In [ ]:
#Logistic Regression
y_pred_lr=classifier_lr.predict(X_t)
cm = confusion_matrix(y_t, y_pred_lr)
print(classification_report(y_t,y_pred_lr))
print(cm)
score_lr=precision_recall_fscore_support(y_t,y_pred_lr,average='weighted')
print(score_lr)

              precision    recall  f1-score   support

           0       0.41      0.12      0.18        59
           1       0.61      0.89      0.72        90

    accuracy                           0.58       149
   macro avg       0.51      0.50      0.45       149
weighted avg       0.53      0.58      0.51       149

[[ 7 52]
 [10 80]]
(0.5291246455873381, 0.5838926174496645, 0.508277086694607, None)


In [ ]:
#Decision Tree
y_pred_dt = classifier_dt.predict(X_t)
cm = confusion_matrix(y_t, y_pred_dt)
print(classification_report(y_t,y_pred_dt))
print(cm)
score_dt=precision_recall_fscore_support(y_t,y_pred_dt,average='weighted')
print(score_dt)

              precision    recall  f1-score   support

           0       0.43      0.27      0.33        59
           1       0.62      0.77      0.68        90

    accuracy                           0.57       149
   macro avg       0.52      0.52      0.51       149
weighted avg       0.54      0.57      0.54       149

[[16 43]
 [21 69]]
(0.5433553160063228, 0.5704697986577181, 0.5446430549095178, None)


In [ ]:
#Random Florest
y_pred_rf = classifier_rf.predict(X_t)
cm = confusion_matrix(y_t, y_pred_rf)
print(classification_report(y_t,y_pred_rf))
print(cm)
score_rf=precision_recall_fscore_support(y_t,y_pred_rf,average='weighted')
print(score_rf)

              precision    recall  f1-score   support

           0       0.50      0.12      0.19        59
           1       0.61      0.92      0.74        90

    accuracy                           0.60       149
   macro avg       0.56      0.52      0.46       149
weighted avg       0.57      0.60      0.52       149

[[ 7 52]
 [ 7 83]]
(0.569351230425056, 0.6040268456375839, 0.5215776408936288, None)


## Test on stormfront

In [ ]:
d = pd.read_csv('/content/gdrive/MyDrive/Colab Notebooks/ML_TEST/Data/ML_SF_Xtest.csv')
# d.head(5)
d.columns.values[0] = 'id'
d.drop('id', inplace=True, axis=1)
X_t = d.to_numpy()
de = pd.read_csv('/content/gdrive/MyDrive/Colab Notebooks/ML_TEST/Data/ML_SF_ytest.csv')
# d.head(5)
de.columns.values[0] = 'id'
de.drop('id', inplace=True, axis=1)
y_t = de.to_numpy()
# X_t

In [ ]:
#Naive Bayes by Davidson or itself
y_pred_np = classifier_np.predict(X_t)
cm = confusion_matrix(y_t, y_pred_np)
print(classification_report(y_t,y_pred_np))
print(cm)
score_np=precision_recall_fscore_support(y_t,y_pred_np,average='weighted')
print(score_np)

              precision    recall  f1-score   support

           0       0.12      0.68      0.20       342
           1       0.91      0.40      0.55      2942

    accuracy                           0.43      3284
   macro avg       0.52      0.54      0.38      3284
weighted avg       0.83      0.43      0.52      3284

[[ 233  109]
 [1774 1168]]
(0.8314816656472708, 0.4266138855054811, 0.5166839509135552, None)


In [ ]:
#KNN
y_pred_knn = classifier_knn.predict(X_t)
cm = confusion_matrix(y_t, y_pred_knn)
print(classification_report(y_t,y_pred_knn))
print(cm)
score_knn=precision_recall_fscore_support(y_t,y_pred_knn,average='weighted')
print(score_knn)

              precision    recall  f1-score   support

           0       0.44      0.02      0.04       342
           1       0.90      1.00      0.94      2942

    accuracy                           0.90      3284
   macro avg       0.67      0.51      0.49      3284
weighted avg       0.85      0.90      0.85      3284

[[   7  335]
 [   9 2933]]
(0.8495867821729777, 0.8952496954933008, 0.8503056057382509, None)


In [ ]:
#XGBoost Classifier
y_pred_xgb = classifier_xgb.predict(X_t)
cm = confusion_matrix(y_t, y_pred_xgb)
print(classification_report(y_t,y_pred_xgb))
print(cm)
score_xgb=precision_recall_fscore_support(y_t,y_pred_xgb,average='weighted')
print(score_xgb)

              precision    recall  f1-score   support

           0       0.74      0.07      0.12       342
           1       0.90      1.00      0.95      2942

    accuracy                           0.90      3284
   macro avg       0.82      0.53      0.54      3284
weighted avg       0.89      0.90      0.86      3284

[[  23  319]
 [   8 2934]]
(0.8852739433373252, 0.9004263093788063, 0.8614144135245133, None)


In [ ]:
#SVM
y_pred_svm = classifier_svm.predict(X_t)
cm = confusion_matrix(y_t, y_pred_svm)
print(classification_report(y_t,y_pred_svm))
print(cm)
score_svm=precision_recall_fscore_support(y_t,y_pred_svm,average='weighted')
print(score_svm)

              precision    recall  f1-score   support

           0       0.70      0.04      0.08       342
           1       0.90      1.00      0.95      2942

    accuracy                           0.90      3284
   macro avg       0.80      0.52      0.51      3284
weighted avg       0.88      0.90      0.86      3284

[[  14  328]
 [   6 2936]]
(0.8787325953523919, 0.8982947624847747, 0.8556997195953079, None)


In [ ]:
#Logistic Regression
y_pred_lr=classifier_lr.predict(X_t)
cm = confusion_matrix(y_t, y_pred_lr)
print(classification_report(y_t,y_pred_lr))
print(cm)
score_lr=precision_recall_fscore_support(y_t,y_pred_lr,average='weighted')
print(score_lr)

              precision    recall  f1-score   support

           0       0.57      0.28      0.38       342
           1       0.92      0.98      0.95      2942

    accuracy                           0.90      3284
   macro avg       0.75      0.63      0.66      3284
weighted avg       0.89      0.90      0.89      3284

[[  96  246]
 [  71 2871]]
(0.8850213640284164, 0.9034713763702802, 0.8882715574157422, None)


In [ ]:
#Decision Tree
y_pred_dt = classifier_dt.predict(X_t)
cm = confusion_matrix(y_t, y_pred_dt)
print(classification_report(y_t,y_pred_dt))
print(cm)
score_dt=precision_recall_fscore_support(y_t,y_pred_dt,average='weighted')
print(score_dt)

              precision    recall  f1-score   support

           0       0.36      0.36      0.36       342
           1       0.93      0.92      0.92      2942

    accuracy                           0.87      3284
   macro avg       0.64      0.64      0.64      3284
weighted avg       0.87      0.87      0.87      3284

[[ 124  218]
 [ 224 2718]]
(0.8664484058703787, 0.8654080389768575, 0.8659245286833444, None)


In [ ]:
#Random Florest
y_pred_rf = classifier_rf.predict(X_t)
cm = confusion_matrix(y_t, y_pred_rf)
print(classification_report(y_t,y_pred_rf))
print(cm)
score_rf=precision_recall_fscore_support(y_t,y_pred_rf,average='weighted')
print(score_rf)

              precision    recall  f1-score   support

           0       0.51      0.20      0.29       342
           1       0.91      0.98      0.94      2942

    accuracy                           0.90      3284
   macro avg       0.71      0.59      0.62      3284
weighted avg       0.87      0.90      0.88      3284

[[  68  274]
 [  65 2877]]
(0.8712031229904448, 0.8967722289890377, 0.8758325611967008, None)


## Test on Reddit

In [ ]:
d = pd.read_csv('/content/gdrive/MyDrive/Colab Notebooks/ML_TEST/Data/ML_RE_Xtest.csv')
# d.head(5)
d.columns.values[0] = 'id'
d.drop('id', inplace=True, axis=1)
X_t = d.to_numpy()
de = pd.read_csv('/content/gdrive/MyDrive/Colab Notebooks/ML_TEST/Data/ML_RE_ytest.csv')
# d.head(5)
de.columns.values[0] = 'id'
de.drop('id', inplace=True, axis=1)
y_t = de.to_numpy()
# X_t

In [ ]:
#Naive Bayes by Davidson or itself
y_pred_np = classifier_np.predict(X_t)
cm = confusion_matrix(y_t, y_pred_np)
print(classification_report(y_t,y_pred_np))
print(cm)
score_np=precision_recall_fscore_support(y_t,y_pred_np,average='weighted')
print(score_np)

              precision    recall  f1-score   support

         0.0       0.26      0.06      0.10      5813
         1.0       0.49      0.84      0.62      6188

    accuracy                           0.46     12001
   macro avg       0.38      0.45      0.36     12001
weighted avg       0.38      0.46      0.37     12001

[[ 356 5457]
 [ 993 5195]]
(0.37929713741156934, 0.4625447879343388, 0.36628482655718125, None)


In [ ]:
#KNN
y_pred_knn = classifier_knn.predict(X_t)
cm = confusion_matrix(y_t, y_pred_knn)
print(classification_report(y_t,y_pred_knn))
print(cm)
score_knn=precision_recall_fscore_support(y_t,y_pred_knn,average='weighted')
print(score_knn)

              precision    recall  f1-score   support

         0.0       0.73      0.00      0.00      5813
         1.0       0.52      1.00      0.68      6188

    accuracy                           0.52     12001
   macro avg       0.62      0.50      0.34     12001
weighted avg       0.62      0.52      0.35     12001

[[   8 5805]
 [   3 6185]]
(0.6182563741039373, 0.5160403299725023, 0.35220897151387653, None)


In [ ]:
#XGBoost Classifier
y_pred_xgb = classifier_xgb.predict(X_t)
cm = confusion_matrix(y_t, y_pred_xgb)
print(classification_report(y_t,y_pred_xgb))
print(cm)
score_xgb=precision_recall_fscore_support(y_t,y_pred_xgb,average='weighted')
print(score_xgb)

              precision    recall  f1-score   support

         0.0       0.80      0.01      0.01      5813
         1.0       0.52      1.00      0.68      6188

    accuracy                           0.52     12001
   macro avg       0.66      0.50      0.35     12001
weighted avg       0.66      0.52      0.36     12001

[[  33 5780]
 [   8 6180]]
(0.6562981716882377, 0.5177068577618532, 0.3566351981259441, None)


In [ ]:
#SVM
y_pred_svm = classifier_svm.predict(X_t)
cm = confusion_matrix(y_t, y_pred_svm)
print(classification_report(y_t,y_pred_svm))
print(cm)
score_svm=precision_recall_fscore_support(y_t,y_pred_svm,average='weighted')
print(score_svm)

              precision    recall  f1-score   support

         0.0       0.00      0.00      0.00      5813
         1.0       0.52      1.00      0.68      6188

    accuracy                           0.52     12001
   macro avg       0.26      0.50      0.34     12001
weighted avg       0.27      0.52      0.35     12001

[[   0 5813]
 [   0 6188]]
(0.26586779796514615, 0.5156236980251646, 0.3508361584891658, None)


/usr/local/lib/python3.7/dist-packages/sklearn/metrics/_classification.py:1318: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.7/dist-packages/sklearn/metrics/_classification.py:1318: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.7/dist-packages/sklearn/metrics/_classification.py:1318: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.7/dist-packages/sklearn/metrics/_classificat

In [ ]:
#Logistic Regression
y_pred_lr=classifier_lr.predict(X_t)
cm = confusion_matrix(y_t, y_pred_lr)
print(classification_report(y_t,y_pred_lr))
print(cm)
score_lr=precision_recall_fscore_support(y_t,y_pred_lr,average='weighted')
print(score_lr)

              precision    recall  f1-score   support

         0.0       0.62      0.08      0.14      5813
         1.0       0.52      0.96      0.68      6188

    accuracy                           0.53     12001
   macro avg       0.57      0.52      0.41     12001
weighted avg       0.57      0.53      0.42     12001

[[ 446 5367]
 [ 271 5917]]
(0.571677630514339, 0.5302058161819848, 0.415404209126864, None)


In [ ]:
#Decision Tree
y_pred_dt = classifier_dt.predict(X_t)
cm = confusion_matrix(y_t, y_pred_dt)
print(classification_report(y_t,y_pred_dt))
print(cm)
score_dt=precision_recall_fscore_support(y_t,y_pred_dt,average='weighted')
print(score_dt)

              precision    recall  f1-score   support

         0.0       0.59      0.27      0.37      5813
         1.0       0.54      0.83      0.66      6188

    accuracy                           0.55     12001
   macro avg       0.57      0.55      0.51     12001
weighted avg       0.57      0.55      0.52     12001

[[1542 4271]
 [1077 5111]]
(0.5660828937581139, 0.5543704691275727, 0.5156771297843656, None)


In [ ]:
#Random Florest
y_pred_rf = classifier_rf.predict(X_t)
cm = confusion_matrix(y_t, y_pred_rf)
print(classification_report(y_t,y_pred_rf))
print(cm)
score_rf=precision_recall_fscore_support(y_t,y_pred_rf,average='weighted')
print(score_rf)

              precision    recall  f1-score   support

         0.0       0.60      0.12      0.20      5813
         1.0       0.53      0.93      0.67      6188

    accuracy                           0.53     12001
   macro avg       0.56      0.52      0.44     12001
weighted avg       0.56      0.53      0.44     12001

[[ 691 5122]
 [ 463 5725]]
(0.5621821222597622, 0.5346221148237646, 0.4426568734743546, None)


## Test on Davidson

In [ ]:
d = pd.read_csv('/content/gdrive/MyDrive/Colab Notebooks/ML_TEST/Data/ML_DAV_Xtest.csv')
# d.head(5)
d.columns.values[0] = 'id'
d.drop('id', inplace=True, axis=1)
X_t = d.to_numpy()
de = pd.read_csv('/content/gdrive/MyDrive/Colab Notebooks/ML_TEST/Data/ML_DAV_ytest.csv')
# d.head(5)
de.columns.values[0] = 'id'
de.drop('id', inplace=True, axis=1)
y_t = de.to_numpy()
# X_t

In [ ]:
#Naive Bayes by Davidson or itself
y_pred_np = classifier_np.predict(X_t)
cm = confusion_matrix(y_t, y_pred_np)
print(classification_report(y_t,y_pred_np))
print(cm)
score_np=precision_recall_fscore_support(y_t,y_pred_np,average='weighted')
print(score_np)

              precision    recall  f1-score   support

           0       0.11      0.20      0.14       427
           1       0.95      0.90      0.93      7008

    accuracy                           0.86      7435
   macro avg       0.53      0.55      0.53      7435
weighted avg       0.90      0.86      0.88      7435

[[  86  341]
 [ 679 6329]]
(0.9008369272910652, 0.8628110289172831, 0.8805664255902985, None)


In [ ]:
#KNN
y_pred_knn = classifier_knn.predict(X_t)
cm = confusion_matrix(y_t, y_pred_knn)
print(classification_report(y_t,y_pred_knn))
print(cm)
score_knn=precision_recall_fscore_support(y_t,y_pred_knn,average='weighted')
print(score_knn)

              precision    recall  f1-score   support

           0       0.00      0.00      0.00       427
           1       0.94      1.00      0.97      7008

    accuracy                           0.94      7435
   macro avg       0.47      0.50      0.49      7435
weighted avg       0.89      0.94      0.91      7435

[[   0  427]
 [   0 7008]]
(0.8884361891831877, 0.9425689307330195, 0.9147023563770685, None)


/usr/local/lib/python3.7/dist-packages/sklearn/metrics/_classification.py:1318: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.7/dist-packages/sklearn/metrics/_classification.py:1318: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.7/dist-packages/sklearn/metrics/_classification.py:1318: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.7/dist-packages/sklearn/metrics/_classificat

In [ ]:
#XGBoost Classifier
y_pred_xgb = classifier_xgb.predict(X_t)
cm = confusion_matrix(y_t, y_pred_xgb)
print(classification_report(y_t,y_pred_xgb))
print(cm)
score_xgb=precision_recall_fscore_support(y_t,y_pred_xgb,average='weighted')
print(score_xgb)

              precision    recall  f1-score   support

           0       0.17      0.00      0.00       427
           1       0.94      1.00      0.97      7008

    accuracy                           0.94      7435
   macro avg       0.55      0.50      0.49      7435
weighted avg       0.90      0.94      0.91      7435

[[   1  426]
 [   5 7003]]
(0.8980911909436985, 0.9420309347679893, 0.9146948919066566, None)


In [ ]:
#SVM
y_pred_svm = classifier_svm.predict(X_t)
cm = confusion_matrix(y_t, y_pred_svm)
print(classification_report(y_t,y_pred_svm))
print(cm)
score_svm=precision_recall_fscore_support(y_t,y_pred_svm,average='weighted')
print(score_svm)

              precision    recall  f1-score   support

           0       0.00      0.00      0.00       427
           1       0.94      1.00      0.97      7008

    accuracy                           0.94      7435
   macro avg       0.47      0.50      0.49      7435
weighted avg       0.89      0.94      0.91      7435

[[   0  427]
 [   1 7007]]
(0.8884289074046633, 0.9424344317417619, 0.9146351610090386, None)


In [ ]:
#Logistic Regression
y_pred_lr=classifier_lr.predict(X_t)
cm = confusion_matrix(y_t, y_pred_lr)
print(classification_report(y_t,y_pred_lr))
print(cm)
score_lr=precision_recall_fscore_support(y_t,y_pred_lr,average='weighted')
print(score_lr)

              precision    recall  f1-score   support

           0       0.09      0.02      0.04       427
           1       0.94      0.99      0.96      7008

    accuracy                           0.93      7435
   macro avg       0.51      0.50      0.50      7435
weighted avg       0.89      0.93      0.91      7435

[[  10  417]
 [ 105 6903]]
(0.8938674116038214, 0.9297915265635508, 0.910348335527909, None)


In [ ]:
#Decision Tree
y_pred_dt = classifier_dt.predict(X_t)
cm = confusion_matrix(y_t, y_pred_dt)
print(classification_report(y_t,y_pred_dt))
print(cm)
score_dt=precision_recall_fscore_support(y_t,y_pred_dt,average='weighted')
print(score_dt)

              precision    recall  f1-score   support

           0       0.08      0.18      0.11       427
           1       0.95      0.87      0.91      7008

    accuracy                           0.83      7435
   macro avg       0.51      0.53      0.51      7435
weighted avg       0.90      0.83      0.86      7435

[[  76  351]
 [ 884 6124]]
(0.8960203146272413, 0.8338937457969066, 0.8625264245483448, None)


In [ ]:
#Random Florest
y_pred_rf = classifier_rf.predict(X_t)
cm = confusion_matrix(y_t, y_pred_rf)
print(classification_report(y_t,y_pred_rf))
print(cm)
score_rf=precision_recall_fscore_support(y_t,y_pred_rf,average='weighted')
print(score_rf)

              precision    recall  f1-score   support

           0       0.12      0.14      0.13       427
           1       0.95      0.94      0.94      7008

    accuracy                           0.89      7435
   macro avg       0.54      0.54      0.54      7435
weighted avg       0.90      0.89      0.90      7435

[[  60  367]
 [ 423 6585]]
(0.8999444797154466, 0.8937457969065232, 0.8968020396889123, None)


## Test on Waseem

In [ ]:
d = pd.read_csv('/content/gdrive/MyDrive/Colab Notebooks/ML_TEST/Data/ML_WS_Xtest.csv')
# d.head(5)
d.columns.values[0] = 'id'
d.drop('id', inplace=True, axis=1)
X_t = d.to_numpy()
de = pd.read_csv('/content/gdrive/MyDrive/Colab Notebooks/ML_TEST/Data/ML_WS_ytest.csv')
# d.head(5)
de.columns.values[0] = 'id'
de.drop('id', inplace=True, axis=1)
y_t = de.to_numpy()
# X_t

In [ ]:
#Naive Bayes by Davidson or itself
y_pred_np = classifier_np.predict(X_t)
cm = confusion_matrix(y_t, y_pred_np)
print(classification_report(y_t,y_pred_np))
print(cm)
score_np=precision_recall_fscore_support(y_t,y_pred_np,average='weighted')
print(score_np)

              precision    recall  f1-score   support

           0       0.16      0.14      0.15       898
           1       0.70      0.73      0.72      2500

    accuracy                           0.57      3398
   macro avg       0.43      0.43      0.43      3398
weighted avg       0.56      0.57      0.57      3398

[[ 124  774]
 [ 671 1829]]
(0.5581790399314569, 0.5747498528546203, 0.5661056777824379, None)


In [ ]:
#KNN
y_pred_knn = classifier_knn.predict(X_t)
cm = confusion_matrix(y_t, y_pred_knn)
print(classification_report(y_t,y_pred_knn))
print(cm)
score_knn=precision_recall_fscore_support(y_t,y_pred_knn,average='weighted')
print(score_knn)

              precision    recall  f1-score   support

           0       0.00      0.00      0.00       898
           1       0.74      1.00      0.85      2500

    accuracy                           0.74      3398
   macro avg       0.37      0.50      0.42      3398
weighted avg       0.54      0.74      0.62      3398

[[   0  898]
 [   0 2500]]
(0.5412940686987914, 0.7357268981753973, 0.6237087980462847, None)


/usr/local/lib/python3.7/dist-packages/sklearn/metrics/_classification.py:1318: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.7/dist-packages/sklearn/metrics/_classification.py:1318: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.7/dist-packages/sklearn/metrics/_classification.py:1318: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.7/dist-packages/sklearn/metrics/_classificat

In [ ]:
#XGBoost Classifier
y_pred_xgb = classifier_xgb.predict(X_t)
cm = confusion_matrix(y_t, y_pred_xgb)
print(classification_report(y_t,y_pred_xgb))
print(cm)
score_xgb=precision_recall_fscore_support(y_t,y_pred_xgb,average='weighted')
print(score_xgb)

              precision    recall  f1-score   support

           0       0.17      0.01      0.02       898
           1       0.73      0.98      0.84      2500

    accuracy                           0.72      3398
   macro avg       0.45      0.50      0.43      3398
weighted avg       0.59      0.72      0.62      3398

[[  10  888]
 [  48 2452]]
(0.5856847932678816, 0.7245438493231312, 0.6233377514337292, None)


In [ ]:
#SVM
y_pred_svm = classifier_svm.predict(X_t)
cm = confusion_matrix(y_t, y_pred_svm)
print(classification_report(y_t,y_pred_svm))
print(cm)
score_svm=precision_recall_fscore_support(y_t,y_pred_svm,average='weighted')
print(score_svm)

              precision    recall  f1-score   support

           0       0.11      0.00      0.00       898
           1       0.73      0.99      0.84      2500

    accuracy                           0.73      3398
   macro avg       0.42      0.50      0.42      3398
weighted avg       0.57      0.73      0.62      3398

[[   2  896]
 [  16 2484]]
(0.5700576469354485, 0.731606827545615, 0.6227681857972622, None)


In [ ]:
#Logistic Regression
y_pred_lr=classifier_lr.predict(X_t)
cm = confusion_matrix(y_t, y_pred_lr)
print(classification_report(y_t,y_pred_lr))
print(cm)
score_lr=precision_recall_fscore_support(y_t,y_pred_lr,average='weighted')
print(score_lr)

              precision    recall  f1-score   support

           0       0.20      0.11      0.14       898
           1       0.73      0.85      0.78      2500

    accuracy                           0.65      3398
   macro avg       0.47      0.48      0.46      3398
weighted avg       0.59      0.65      0.61      3398

[[  95  803]
 [ 370 2130]]
(0.5882900491072816, 0.6547969393761036, 0.6137206327960668, None)


In [ ]:
#Decision Tree
y_pred_dt = classifier_dt.predict(X_t)
cm = confusion_matrix(y_t, y_pred_dt)
print(classification_report(y_t,y_pred_dt))
print(cm)
score_dt=precision_recall_fscore_support(y_t,y_pred_dt,average='weighted')
print(score_dt)

              precision    recall  f1-score   support

           0       0.19      0.32      0.24       898
           1       0.68      0.51      0.58      2500

    accuracy                           0.46      3398
   macro avg       0.43      0.42      0.41      3398
weighted avg       0.55      0.46      0.49      3398

[[ 287  611]
 [1214 1286]]
(0.5492890224495144, 0.46291936433195996, 0.49359090424985275, None)


In [ ]:
#Random Florest
y_pred_rf = classifier_rf.predict(X_t)
cm = confusion_matrix(y_t, y_pred_rf)
print(classification_report(y_t,y_pred_rf))
print(cm)
score_rf=precision_recall_fscore_support(y_t,y_pred_rf,average='weighted')
print(score_rf)

              precision    recall  f1-score   support

           0       0.19      0.30      0.23       898
           1       0.68      0.54      0.60      2500

    accuracy                           0.48      3398
   macro avg       0.44      0.42      0.42      3398
weighted avg       0.55      0.48      0.51      3398

[[ 269  629]
 [1144 1356]]
(0.5529032941944214, 0.4782224838140082, 0.5064037006349706, None)
